In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    !pip install -q tf-nightly
except Exception:
    pass
import tensorflow as tf

import numpy as np
import IPython.display as display

tf.__version__

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
ERROR: Could not find a version that satisfies the requirement tf-nightly (from versions: none)
ERROR: No matching distribution found for tf-nightly


/home/duyvc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/duyvc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/duyvc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/duyvc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

'1.14.0'

**Cách load dữ liệu truyền thống**
<img src="./images/traditional_load_data.png">

**Cách load dữ liệu bằng cách Pipeline**
<img src="./images/pipeline_load_data.png">
Trong quá trình làm việc với TensorFlow, cách dễ dàng nhất để cấu hình một input pipeline hiệu quả là sử dụng định dạng TFRecord, một định dạng dữ liệu kiểu nhị phân được hỗ trợ bởi TensorFlow.

### Protocol Buffers & TFRecord
**Protocol Buffers** định nghĩa cách serialize những dữ liệu có cấu trúc thành dữ liệu nhị phân. <br>
**TFRecord** là một định dạng đơn giản để lưu trữ một chuỗi các bản ghi nhị phân. 1 tệp TFRecord chứa một chuỗi các records của observation. Các tập chỉ có thể được đọc tuần tự.<br>

Do đó, TensorFlow sử dụng Protocol Buffers để định nghĩa cấu trúc của dữ liệu dạng TFRecord. <br>
Một trường dữ liệu (Ví dụ: ảnh đầu vào và nhãn của ảnh đó) được gọi là một Example<br>

### 1 tf.Example là 1 ánh xạ {"string": tf.train.Feature}.** <br>
**Data types for tf.Example**

<img src="./images/Tf_Feature_datatype.png">

In order to convert a standard TensorFlow type to a tf.Example-compatible tf.train.Feature, you can use the shortcut functions below. Note that each function takes a scalar input value and returns a tf.train.Feature containing one of the three list types above:

In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_bytes"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 1
}

int64_list {
  value: 1
}



In [4]:
feature = _float_feature(np.exp(1))
feature.SerializeToString()

b'\x12\x06\n\x04T\xf8-@'

### Example

In this notebook, you will create a dataset using NumPy.

This dataset will have 4 features:
- a boolean feature, False or True with equal probability
- an integer feature uniformly randomly chosen from [0, 5]
- a string feature generated from a string table by using the integer feature as an index
- a float feature from a standard normal distribution

Consider a sample consisting of 10,000 independently and identically distributed observations from each of the above distributions:

In [5]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)

### Tổng quát quá trình tạo 1 tf.Example message

1. Within each observation, each value needs to be converted to a tf.train.Feature containing one of the 3 compatible types, using one of the functions above.
2. You create a map (dictionary) from the feature name string to the encoded feature value produced in #1.
3. The map produced in step 2 is converted to a Features message.

In [6]:
# Serialize a sample from observation to a Example
def serialize_sample(feature0, feature1, feature2, feature3):
    feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

example = serialize_sample(False, 4, b'goat', 0.9876)
print(example)

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "feature2"
    value {
      bytes_list {
        value: "goat"
      }
    }
  }
  feature {
    key: "feature3"
    value {
      float_list {
        value: 0.9876000285148621
      }
    }
  }
}



### TFRecord files using tf.data
**Writing a TFRecord file**<br>
Cách đơn giản nhất để tạo dataset từ các features là sử dụng method **from_tensor_slices**<br>
Từ đó có thể can thiệp để tiền xử lý thông qua method **tf.data.Dataset.map**, áp dụng cho mỗi element trong Dataset.


In [10]:
tf.data.Dataset.from_tensor_slices(feature1)

<DatasetV1Adapter shapes: (), types: tf.int64>

In [7]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

<DatasetV1Adapter shapes: ((), (), (), ()), types: (tf.bool, tf.int64, tf.string, tf.float64)>

In [9]:
# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
    print(f0)
    print(f1)
    print(f2)
    print(f3)
    break

Tensor("IteratorGetNext_50041:0", shape=(), dtype=bool)
Tensor("IteratorGetNext_50041:1", shape=(), dtype=int64)
Tensor("IteratorGetNext_50041:2", shape=(), dtype=string)
Tensor("IteratorGetNext_50041:3", shape=(), dtype=float64)
